# Read the training data

## Read specific columns identified from EDA

In [4]:
import pandas as pd
import numpy as np

In [5]:
# Read specific columns of the csv files - we do this instead of reading all the columns to save memory

features_to_retain = pd.read_csv('columns_to_retain.csv')
features_to_retain_identity = pd.read_csv('columns_to_retain_identity.csv')

print(features_to_retain.head())
print(features_to_retain_identity.head())


   Unnamed: 0 columns_to_retain
0           0               V27
1           1               V75
2           2               V44
3           3               V39
4           4              V224
   Unnamed: 0 columns_to_retain
0           0     TransactionID
1           1             id_01
2           2             id_02
3           3             id_03
4           4             id_04


In [6]:
train_transaction_data = pd.read_csv("train_transaction.csv",usecols = features_to_retain['columns_to_retain'].values)
train_identity_data = pd.read_csv("train_identity.csv",usecols = features_to_retain_identity['columns_to_retain'].values) 

In [7]:
train_transaction_data.shape

(590540, 114)

## Merge train_transaction_data and train_identity_data based on TransactionID

In [8]:
all_data = pd.merge(train_transaction_data,train_identity_data,how = "left", on = 'TransactionID')
all_data.head()

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
0,2987000,0,86400,68.5,W,13926,NaN,150.0,discover,142.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2987001,0,86401,29.0,W,2755,404.0,150.0,mastercard,102.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2987002,0,86469,59.0,W,4663,490.0,150.0,visa,166.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2987003,0,86499,50.0,W,18132,567.0,150.0,mastercard,117.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2987004,0,86506,50.0,H,4497,514.0,150.0,mastercard,102.0,...,samsung browser 6.2,32.0,2220x1080,match_status:2,T,F,T,T,mobile,SAMSUNG SM-G892A Build/NRD90M


In [9]:
all_data['addr1']

0         315.0
1         325.0
2         330.0
3         476.0
4         420.0
          ...  
590535    272.0
590536    204.0
590537    231.0
590538    387.0
590539    299.0
Name: addr1, Length: 590540, dtype: float64

In [10]:
print('There are a total of:'+str(all_data.shape[0])+' rows of data')

There are a total of:590540 rows of data


In [11]:
# List of categorical and numerical columns in merged dataset

## Numerical columns

num_column_lst = []
for icol in all_data.columns:
  if all_data[icol].dtypes in ['float64', 'int64']:
    num_column_lst.append(icol)
print(f"The columns that have numerical features are: '{num_column_lst}'")
print(f"The number of columns that have numerical features are: '{len(num_column_lst)}'")

print('\n')

## Categorical COlumns
cat_column_lst = []
for icol in all_data.columns:
  if all_data[icol].dtypes in ['object']:
    cat_column_lst.append(icol)
print(f"The columns that have numerical features are: '{cat_column_lst}'")
print(f"The number of columns that have numerical features are: '{len(cat_column_lst)}'")


The columns that have numerical features are: '['TransactionID', 'isFraud', 'TransactionDT', 'TransactionAmt', 'card1', 'card2', 'card3', 'card5', 'addr1', 'addr2', 'dist1', 'dist2', 'C1', 'C3', 'C5', 'D1', 'D3', 'D4', 'D8', 'D9', 'D10', 'D13', 'D14', 'V1', 'V2', 'V4', 'V6', 'V10', 'V12', 'V14', 'V15', 'V19', 'V23', 'V25', 'V27', 'V35', 'V37', 'V39', 'V44', 'V46', 'V53', 'V55', 'V61', 'V66', 'V75', 'V77', 'V82', 'V86', 'V95', 'V98', 'V99', 'V104', 'V107', 'V108', 'V109', 'V117', 'V118', 'V120', 'V121', 'V123', 'V124', 'V129', 'V135', 'V138', 'V139', 'V141', 'V144', 'V148', 'V161', 'V169', 'V170', 'V172', 'V173', 'V174', 'V184', 'V194', 'V208', 'V209', 'V214', 'V220', 'V221', 'V223', 'V224', 'V226', 'V227', 'V228', 'V238', 'V240', 'V241', 'V250', 'V260', 'V270', 'V281', 'V282', 'V284', 'V286', 'V288', 'V290', 'V300', 'V305', 'V313', 'id_01', 'id_02', 'id_03', 'id_04', 'id_05', 'id_06', 'id_07', 'id_08', 'id_10', 'id_11', 'id_13', 'id_14', 'id_17', 'id_18', 'id_19', 'id_20', 'id_21', 'id

### Summary of the Merged Dataframe

The dataframe that has been created with the processed features from **1_EDA_Loading_Data_and_train_transaction** and **2_EDA_train_identity** contains 590540 observations and   features. This datframe will be utilized for conducting feature engineering, to transform null values, label encoding the categorical features and scaling certain numerical features rest of which would be scaled dusing the phase of model building. 

## Feature Engineering

### Replacing Nan values in Categorical features 

In [12]:
# replace nan values as 'missing' in categorical columns

all_data[cat_column_lst] = all_data[cat_column_lst].replace({ np.nan:'missing'})

### Replacing NAN values in Numerical Variables

In [13]:
all_data[num_column_lst] = all_data[num_column_lst].replace({ np.nan:-1})

id_30, id_31, id_33 and DeviceInfo are columns with a lot if uniques featurea so we handle them separately before label encoding them

### Column id_30

In [14]:
# Column id_30
np.unique(all_data['id_30'])


array(['Android', 'Android 4.4.2', 'Android 5.0', 'Android 5.0.2',
       'Android 5.1.1', 'Android 6.0', 'Android 6.0.1', 'Android 7.0',
       'Android 7.1.1', 'Android 7.1.2', 'Android 8.0.0', 'Android 8.1.0',
       'Linux', 'Mac', 'Mac OS X 10.10', 'Mac OS X 10.11',
       'Mac OS X 10.12', 'Mac OS X 10.13', 'Mac OS X 10.6',
       'Mac OS X 10.9', 'Mac OS X 10_10_5', 'Mac OS X 10_11_3',
       'Mac OS X 10_11_4', 'Mac OS X 10_11_5', 'Mac OS X 10_11_6',
       'Mac OS X 10_12', 'Mac OS X 10_12_1', 'Mac OS X 10_12_2',
       'Mac OS X 10_12_3', 'Mac OS X 10_12_4', 'Mac OS X 10_12_5',
       'Mac OS X 10_12_6', 'Mac OS X 10_13_1', 'Mac OS X 10_13_2',
       'Mac OS X 10_13_3', 'Mac OS X 10_13_4', 'Mac OS X 10_13_5',
       'Mac OS X 10_6_8', 'Mac OS X 10_7_5', 'Mac OS X 10_8_5',
       'Mac OS X 10_9_5', 'Windows', 'Windows 10', 'Windows 7',
       'Windows 8', 'Windows 8.1', 'Windows Vista', 'Windows XP', 'func',
       'iOS', 'iOS 10.0.2', 'iOS 10.1.1', 'iOS 10.2.0', 'iOS 10.2.1',

In [15]:
# id_30 COlumn (categorical) feature engineering
all_data.loc[all_data['id_30'].str.contains('Mac', na=False), 'id_30'] = 'mac'
all_data.loc[all_data['id_30'].str.contains('iOS', na=False), 'id_30'] = 'iOS'
all_data.loc[all_data['id_30'].str.contains('Android', na=False), 'id_30'] = 'android'
all_data.loc[all_data['id_30'].str.contains('Windows', na=False), 'id_30'] = 'Windows'
all_data.loc[all_data['id_30'].str.contains('Linux', na=False), 'id_30'] = 'Linux'

### Column id_31

In [16]:
# Column id_31
np.unique(all_data['id_31'])

array(['BLU/Dash', 'Cherry', 'Generic/Android', 'Generic/Android 7.0',
       'Inco/Minion', 'LG/K-200', 'Lanix/Ilium', 'M4Tel/M4',
       'Microsoft/Windows', 'Mozilla/Firefox', 'Nokia/Lumia',
       'Samsung/SCH', 'Samsung/SM-G531H', 'Samsung/SM-G532M', 'ZTE/Blade',
       'android', 'android browser 4.0', 'android webview 4.0', 'aol',
       'chrome', 'chrome 43.0 for android', 'chrome 46.0 for android',
       'chrome 49.0', 'chrome 49.0 for android',
       'chrome 50.0 for android', 'chrome 51.0',
       'chrome 51.0 for android', 'chrome 52.0 for android',
       'chrome 53.0 for android', 'chrome 54.0 for android',
       'chrome 55.0', 'chrome 55.0 for android', 'chrome 56.0',
       'chrome 56.0 for android', 'chrome 57.0',
       'chrome 57.0 for android', 'chrome 58.0',
       'chrome 58.0 for android', 'chrome 59.0',
       'chrome 59.0 for android', 'chrome 60.0',
       'chrome 60.0 for android', 'chrome 61.0',
       'chrome 61.0 for android', 'chrome 62.0',
       'chr

In [17]:
# id_31 COlumn (categorical) feature engineering

print(np.unique(all_data['id_31']))
all_data['id_31'] = all_data['id_31'].str.replace('\d+', '')
print(f"The reduced uniques values are:\n\n '{np.unique(all_data['id_31'])}'")

['BLU/Dash' 'Cherry' 'Generic/Android' 'Generic/Android 7.0' 'Inco/Minion'
 'LG/K-200' 'Lanix/Ilium' 'M4Tel/M4' 'Microsoft/Windows' 'Mozilla/Firefox'
 'Nokia/Lumia' 'Samsung/SCH' 'Samsung/SM-G531H' 'Samsung/SM-G532M'
 'ZTE/Blade' 'android' 'android browser 4.0' 'android webview 4.0' 'aol'
 'chrome' 'chrome 43.0 for android' 'chrome 46.0 for android'
 'chrome 49.0' 'chrome 49.0 for android' 'chrome 50.0 for android'
 'chrome 51.0' 'chrome 51.0 for android' 'chrome 52.0 for android'
 'chrome 53.0 for android' 'chrome 54.0 for android' 'chrome 55.0'
 'chrome 55.0 for android' 'chrome 56.0' 'chrome 56.0 for android'
 'chrome 57.0' 'chrome 57.0 for android' 'chrome 58.0'
 'chrome 58.0 for android' 'chrome 59.0' 'chrome 59.0 for android'
 'chrome 60.0' 'chrome 60.0 for android' 'chrome 61.0'
 'chrome 61.0 for android' 'chrome 62.0' 'chrome 62.0 for android'
 'chrome 62.0 for ios' 'chrome 63.0' 'chrome 63.0 for android'
 'chrome 63.0 for ios' 'chrome 64.0' 'chrome 64.0 for android'
 'chrome 6

<ipython-input-17-4729fb40cf60>:4: FutureWarning: The default value of regex will change from True to False in a future version.
  all_data['id_31'] = all_data['id_31'].str.replace('\d+', '')


The reduced uniques values are:

 '['BLU/Dash' 'Cherry' 'Generic/Android' 'Generic/Android .' 'Inco/Minion'
 'LG/K-' 'Lanix/Ilium' 'MTel/M' 'Microsoft/Windows' 'Mozilla/Firefox'
 'Nokia/Lumia' 'Samsung/SCH' 'Samsung/SM-GH' 'Samsung/SM-GM' 'ZTE/Blade'
 'android' 'android browser .' 'android webview .' 'aol' 'chrome'
 'chrome .' 'chrome . for android' 'chrome . for ios' 'chrome generic'
 'chrome generic for android' 'chromium' 'comodo' 'cyberfox' 'edge'
 'edge .' 'facebook' 'firefox' 'firefox .' 'firefox generic'
 'firefox mobile .' 'google' 'google search application .' 'icedragon'
 'ie' 'ie . for desktop' 'ie . for tablet' 'iron' 'line' 'maxthon'
 'missing' 'mobile' 'mobile safari .' 'mobile safari generic'
 'mobile safari uiwebview' 'opera' 'opera .' 'opera generic' 'other'
 'palemoon' 'puffin' 'safari' 'safari .' 'safari generic' 'samsung'
 'samsung browser .' 'samsung browser generic' 'seamonkey' 'silk'
 'waterfox']'


In [18]:
# id_31 COlumn (categorical) feature engineering
all_data.loc[all_data['id_31'].str.contains('chrome', na=False), 'id_31'] = 'chrome'
all_data.loc[all_data['id_31'].str.contains('edge', na=False), 'id_31'] = 'iOS'
all_data.loc[all_data['id_31'].str.contains('|'.join(['Android','android']), na=False), 'id_31'] = 'android'
all_data.loc[all_data['id_31'].str.contains('google', na=False), 'id_31'] = 'google'
all_data.loc[all_data['id_31'].str.contains('safari', na=False), 'id_31'] = 'safari'
all_data.loc[all_data['id_31'].str.contains('ie', na=False), 'id_31'] = 'ie'
all_data.loc[all_data['id_31'].str.contains('|'.join(['samsung','Samsung']), na=False), 'id_31'] = 'samsung'
all_data.loc[all_data['id_31'].str.contains('|'.join (['firefox','Firefox']), na=False), 'id_31'] = 'firefox'
all_data.loc[all_data['id_31'].str.contains('opera', na=False), 'id_31'] = 'opera'



print(f"The reduced uniques values are:\n\n '{np.unique(all_data['id_31'])}'")


The reduced uniques values are:

 '['BLU/Dash' 'Cherry' 'Inco/Minion' 'LG/K-' 'Lanix/Ilium' 'MTel/M'
 'Microsoft/Windows' 'Nokia/Lumia' 'ZTE/Blade' 'android' 'aol' 'chrome'
 'chromium' 'comodo' 'cyberfox' 'facebook' 'firefox' 'google' 'iOS'
 'icedragon' 'ie' 'iron' 'line' 'maxthon' 'missing' 'mobile' 'opera'
 'other' 'palemoon' 'puffin' 'safari' 'samsung' 'seamonkey' 'silk'
 'waterfox']'


### Column id_33

In [19]:
# Column id_33
np.unique(all_data['id_33'])

array(['0x0', '1023x767', '1023x768', '1024x552', '1024x600', '1024x640',
       '1024x767', '1024x768', '1024x819', '1025x768', '1062x630',
       '1092x614', '1093x615', '1120x700', '1136x640', '1138x640',
       '1152x648', '1152x720', '1152x864', '1184x720', '1188x720',
       '1200x675', '1200x720', '1229x691', '1232x800', '1239x697',
       '1264x924', '1272x960', '1279x1023', '1279x1024', '1280x1023',
       '1280x1024', '1280x1025', '1280x600', '1280x620', '1280x712',
       '1280x720', '1280x732', '1280x740', '1280x768', '1280x800',
       '1280x900', '1280x960', '1281x720', '1281x721', '1281x800',
       '1281x801', '1296x774', '1334x750', '1344x756', '1344x840',
       '1356x900', '1359x768', '1360x767', '1360x768', '1364x768',
       '1365x767', '1365x768', '1366x1024', '1366x767', '1366x768',
       '1368x768', '1371x857', '1400x1050', '1400x900', '1408x792',
       '1408x880', '1422x889', '1439x809', '1439x899', '1439x900',
       '1440x720', '1440x759', '1440x800', '1440

Upon examining the column, it appears to relate to resolution. Considering that resolution may not play a crucial role in fraud detection, we have opted to drop this column from the dataset.

In [20]:
# drop the column id_33
all_data = all_data.drop('id_33', axis=1)  


### Column Deviceinfo

In [21]:
# Column Deviceinfo
print(len(np.unique(all_data['DeviceInfo'])))
np.unique(all_data['DeviceInfo'])


1787


array(['0PAJ5', '0PJA2', '0PM92', ..., 'verykools5035', 'vivo',
       'xs-Z47b7VqTMxs'], dtype=object)

In [22]:
all_data['DeviceInfo']

0                               missing
1                               missing
2                               missing
3                               missing
4         SAMSUNG SM-G892A Build/NRD90M
                      ...              
590535                          missing
590536                          missing
590537                          missing
590538                          missing
590539                          missing
Name: DeviceInfo, Length: 590540, dtype: object

In [23]:
all_data['DeviceInfo'].str.split('/', expand=True)[0]

0                        missing
1                        missing
2                        missing
3                        missing
4         SAMSUNG SM-G892A Build
                   ...          
590535                   missing
590536                   missing
590537                   missing
590538                   missing
590539                   missing
Name: 0, Length: 590540, dtype: object

This column could be directly label encoded

In [24]:
np.unique(all_data['DeviceInfo'].str.split('/', expand=True)[0])


array(['0PAJ5', '0PJA2', '0PM92', ..., 'verykools5035', 'vivo',
       'xs-Z47b7VqTMxs'], dtype=object)

In [25]:
print(len(np.unique(all_data['DeviceInfo'].str.split('/', expand=True)[0])))


1575


### Categorical Variables of transaction data with a lot of unique or string values requiring preprocessing

In [26]:
all_data['P_emaildomain']=all_data['P_emaildomain'].str.split('.',expand=True)[0]
all_data['R_emaildomain']=all_data['R_emaildomain'].str.split('.',expand=True)[0]

### Numerical Variables of Transaction Amount requiring preprocessing

In [27]:
all_data['TransactionAmt'] = all_data['TransactionAmt'].apply(np.log10)

### Combining features and creating new features

In [28]:
# combine primary email and address and card 1

all_data['P_emaildomain_addr1_card1'] = all_data['P_emaildomain'].astype(str) + \
'_' + all_data['addr1'].astype(str) + '_' + all_data['card1'].astype(str)

In [29]:
# combine card 1 and card 2

all_data['card1_card2'] = all_data['card1'].astype(str) + \
'_' + all_data['card2'].astype(str)

In [30]:
# combine addr 1 and card 2

all_data['card2_addr1'] = all_data['card2'].astype(str) + \
'_' + all_data['addr1'].astype(str)

In [31]:
# combine card 2 and secondary email

all_data['card2_R_emaildomain'] = all_data['card2'].astype(str) + \
'_' + all_data['R_emaildomain'].astype(str)

In [32]:
# combine card 1 and secondary email

all_data['card1_R_emaildomain'] = all_data['card1'].astype(str) + \
'_' + all_data['R_emaildomain'].astype(str)

### Label encoding the categorical variables

In [33]:
# List of categorical and numerical columns in merged dataset

## Numerical columns

num_column_lst_final = []
for icol in all_data.columns:
  if all_data[icol].dtypes in ['float64', 'int64']:
    num_column_lst_final.append(icol)
print(f"The columns that have numerical features are: '{num_column_lst}'")
print(f"The number of columns that have numerical features are: '{len(num_column_lst)}'")

print('\n')

## Categorical Columns
cat_column_lst_final = []
for icol in all_data.columns:
  if all_data[icol].dtypes in ['object']:
    cat_column_lst_final.append(icol)
print(f"The columns that have categorical features are: '{cat_column_lst}'")
print(f"The number of columns that have categorical features are: '{len(cat_column_lst)}'")


The columns that have numerical features are: '['TransactionID', 'isFraud', 'TransactionDT', 'TransactionAmt', 'card1', 'card2', 'card3', 'card5', 'addr1', 'addr2', 'dist1', 'dist2', 'C1', 'C3', 'C5', 'D1', 'D3', 'D4', 'D8', 'D9', 'D10', 'D13', 'D14', 'V1', 'V2', 'V4', 'V6', 'V10', 'V12', 'V14', 'V15', 'V19', 'V23', 'V25', 'V27', 'V35', 'V37', 'V39', 'V44', 'V46', 'V53', 'V55', 'V61', 'V66', 'V75', 'V77', 'V82', 'V86', 'V95', 'V98', 'V99', 'V104', 'V107', 'V108', 'V109', 'V117', 'V118', 'V120', 'V121', 'V123', 'V124', 'V129', 'V135', 'V138', 'V139', 'V141', 'V144', 'V148', 'V161', 'V169', 'V170', 'V172', 'V173', 'V174', 'V184', 'V194', 'V208', 'V209', 'V214', 'V220', 'V221', 'V223', 'V224', 'V226', 'V227', 'V228', 'V238', 'V240', 'V241', 'V250', 'V260', 'V270', 'V281', 'V282', 'V284', 'V286', 'V288', 'V290', 'V300', 'V305', 'V313', 'id_01', 'id_02', 'id_03', 'id_04', 'id_05', 'id_06', 'id_07', 'id_08', 'id_10', 'id_11', 'id_13', 'id_14', 'id_17', 'id_18', 'id_19', 'id_20', 'id_21', 'id

In [34]:
# Label encoding the categorical variable

from sklearn import preprocessing
for icol in cat_column_lst_final:
  le = preprocessing.LabelEncoder()
  all_data[icol] = le.fit_transform(all_data[icol])

all_data.head()


,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,id_36,id_37,id_38,DeviceType,DeviceInfo,P_emaildomain_addr1_card1,card1_card2,card2_addr1,card2_R_emaildomain,card1_R_emaildomain
0,2987000,0,86400,1.835691,4,13926,-1.0,150.0,1,142.0,...,2,2,2,1,1735,63363,3655,33,14,7070
1,2987001,0,86401,1.462398,4,2755,404.0,150.0,2,102.0,...,2,2,2,1,1735,36911,8484,4426,2305,16599
2,2987002,0,86469,1.770852,4,4663,490.0,150.0,4,166.0,...,2,2,2,1,1735,71571,10055,5848,2911,19628
3,2987003,0,86499,1.698970,4,18132,567.0,150.0,2,117.0,...,2,2,2,1,1735,87778,7483,7280,3603,14644
4,2987004,0,86506,1.698970,1,4497,514.0,150.0,2,102.0,...,0,1,1,2,954,40159,9929,6249,3128,19374


## Save the feature-engineered DataFrame as a `**.csv**` file

In [35]:
# Converting the dataframe as .csv file

pd.DataFrame(all_data).to_csv('df.csv')


In [36]:
# Copying and saving the .csv file in the google drive 

!cp 'df.csv' '/content/drive/MyDrive/PCMALAI_UCBerkeley_Capstone/Classification_products/df.csv'

## Summary

To prepare the dataframe for model building, we implemented various feature engineering techniques. This included:

   - Label encoding of categorical columns.
   - Creation of new features by combining existing ones, resulting in the addition of five new features to the dataframe.
   - Removal of one feature (e.g., id_33).
   - Derivation and consolidation of unique values within the columns device info, id_31, and id_30.
   - Replacement of NaN values in both numerical and categorical features.

Starting with a dataframe that merged both transaction and identity data, containing 153 features, we enhanced it for model building purposes through feature engineering. The resulting dataframe now comprises 157 features, while maintaining the same number of rows (590540) as the initial dataframe.